## About this notebook:

This notebook uses saved transformations to align all channels in a selected well (works for a single well because manual checking of alignment for each well is a good idea).

Input:
- a set of transforms (one transform/round)
- data frames with channel data for each channel
- nd2 images

Output:
- aligned tiffs 

## Fill in info about the experiment to process

In [2]:
# pathway to a directory with data frames (ex. df)
path_df = r''
path_tmat=path_df

# pathway to save aligned tiffs (ex. aligned_tiffs)
path_save = r''

# list of wells to be processed (usually names as 'A3')
selected_well = ''

# if you want to use this notebook for visualization of results
collect_for_test = True

## Prepare for processing

In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import nd2

from tifffile import imsave
from skimage import transform

import ipywidgets as widgets
from IPython.display import display

In [3]:
# create directory for saving data frames if needed
try:
    os.mkdir(path_save)
    print('Directory for saving aligned tiffs created.')
except:
    print('Directory not created.')
    
try:
    os.mkdir(os.path.join(path_save,selected_well))
    print('Sub-directory for saving aligned tiffs created.')
except:
    print('Directory not created.')

Directory not created.
Directory not created.


In [4]:
df_name = f'df_{selected_well}.pkl'
tmat_name = f'tmat_{selected_well}.pkl'

In [8]:
# read in the data
#myData = pd.read_pickle(os.path.join(path_df,df_name))

tmat_list = pickle.load(open(os.path.join(path_tmat,tmat_name), "rb"))

In [7]:
import pickle5 as pickle
with open(os.path.join(path_df,df_name), "rb") as fh:
  myData = pickle.load(fh)

# Align and save images

In [11]:
myProgress = widgets.FloatProgress()
myProgress.min = 0
myProgress.max = len(myData)
myProgress.value = 0
display(myProgress)

test_im = []
for ind,my_signal in myData.iterrows():
    
    # set progress bar
    myProgress.value = ind
    
    # get a path to the image file
    file_path = os.path.join(my_signal.dir,my_signal.sub_dir,my_signal.file)

    # get a handle to the nd2 file
    myIm = nd2.ND2File(file_path)

    # choose the right frame
    dask_im = myIm.to_dask()
    im = dask_im[my_signal.channel_in_file,:,:]
    
    # trim if needed
    if ((my_signal.width_min < my_signal.width) | (my_signal.height_min < my_signal.height)):

        im = im[:my_signal.height_min,:my_signal.width_min]
    
    # get the transform
    tf = tmat_list[int(my_signal.alignRound)]
    
    # apply transform
    im_alig = transform.warp(im,tf,output_shape=im.shape)
    im_alig[im_alig<0] = 0
    
    if collect_for_test == True:
        test_im.append(im_alig)
    
    # save image
    imsave(os.path.join(path_save,selected_well,f'Round_{str(int(my_signal.nameRound)).zfill(2)}_well{my_signal.well}_{my_signal.signal}.tif'),im_alig.astype('uint16'))
    
myProgress.value = len(myData)

FloatProgress(value=0.0, max=13.0)

## Optional - view alignment

In [12]:
import napari

In [17]:
# test alignment

test_im = np.array(test_im)

viewer = napari.Viewer()
viewer.add_image(test_im,blending = 'additive')

<Image layer 'Image' at 0x1d8da560548>